In [ ]:
import numpy as np
from scipy.optimize import fsolve
import matplotlib.pyplot as plt

# Objective function G(v)
def G(v):
    Q = np.array([[1, 3], [5, 7]])  # Quadratic matrix
    r = np.array([1, 1])  # Linear term
    c = 2  # Constant term
    return np.einsum('ij,ij->i', v @ Q, v) + r @ v.T + c

# Constraint function c1(v) = 2*v1 - 5*v2 - 2
def c1(v):
    return np.array([2 * v[0] - 5 * v[1] - 2])

# Constraint function c2(v) = v1 + v2 - 1
def c2(v):
    return np.array([v[0] + v[1] - 1])

# Lagrangian function
def lagrangian(v, *lmbda):
    l1, l2 = lmbda
    return G(v) - l1 * c1(v) - l2 * c2(v)

# Lagrangian gradient with respect to v
def lagrangian_grad(v, l1, l2):
    Q = np.array([[1, 3], [5, 7]])
    r = np.array([1, 1])
    return 2 * Q @ v + r - 2 * l1 * np.array([2, -5]) - l2 * np.array([1, 1])

# Initial guess for decision variables and Lagrange multipliers
initial_guess = np.array([0.5, 0.5, 1, 1])

# Function to solve system equations (gradient + constraints)
def eq_system(vars):
    v = vars[:2]  # v1, v2
    l1, l2 = vars[2:]  # Lagrange multipliers
    return np.concatenate((lagrangian_grad(v, l1, l2), c1(v), c2(v)))

# Solve using fsolve
solution = fsolve(eq_system, initial_guess)

# Extract solution values
v_opt = solution[:2]  # Optimal v1, v2
l1_opt, l2_opt = solution[2], solution[3]  # Optimal multipliers

# Print results
print("Optimal v1, v2:", v_opt)
print("Optimal lambda1:", l1_opt)
print("Optimal lambda2:", l2_opt)

# Set up grid for plotting
v1_vals = np.linspace(-2, 2, 100)
v2_vals = np.linspace(-2, 2, 100)
V1, V2 = np.meshgrid(v1_vals, v2_vals)

# Compute G(v) over grid for plotting
G_vals = G(np.array([V1.flatten(), V2.flatten()]).T).reshape(V1.shape)

# Create 3D plot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(V1, V2, G_vals, cmap='viridis', alpha=0.6)
ax.scatter(v_opt[0], v_opt[1], G(np.array([v_opt]))[0], color='red')  # Optimal point

# Plot constraint surfaces in 3D
ax.plot_surface(V1, V2, 2 * V1 - 5 * V2 - 2, color='blue', alpha=0.3)
ax.plot_surface(V1, V2, V1 + V2 - 1, color='orange', alpha=0.3)

# Axis labels
ax.set_xlabel('v1')
ax.set_ylabel('v2')
ax.set_zlabel('G(v)')
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import torch
from torchvision import datasets, transforms

# Load and preprocess the Fashion MNIST dataset
transformations = transforms.Compose([transforms.ToTensor(), transforms.Lambda(lambda img: img.view(-1))])
fashion_data = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transformations)

# Convert the dataset into a numpy array
features = torch.stack([img for img, _ in fashion_data]).numpy()

# Normalize the data before PCA
data_scaler = StandardScaler()
normalized_features = data_scaler.fit_transform(features)

# Perform PCA on the normalized data
pca_model = PCA(n_components=40)
pca_model.fit(normalized_features)

# Compute cumulative variance for each component
cum_variance = np.cumsum(pca_model.explained_variance_ratio_)

# Find the number of components that explain at least 50% of the variance
num_components_50_var = np.argmax(cum_variance >= 0.50) + 1

# Display the number of components for 50% variance
print(f"Number of components to reach 50% variance: {num_components_50_var}")

# Plot cumulative variance for the first 40 components
plt.figure(figsize=(10, 6))
plt.plot(np.arange(1, 41), cum_variance, marker='o', linestyle='--')
plt.axhline(y=0.50, color='r', linestyle='-', label='50% variance')
plt.axvline(x=num_components_50_var, color='g', linestyle='-', label=f'{num_components_50_var} components')
plt.title('Cumulative Explained Variance for First 40 PCA Components')
plt.xlabel('Number of PCA Components')
plt.ylabel('Cumulative Explained Variance')
plt.grid(True)
plt.legend()
plt.show()


